In [1]:
import pandas as pd
import os
import numpy as np
import datetime
import matplotlib.pyplot as plt
#import geopandas as gpd
import shapely as shp
#import descartes
#from shapely.geometry import Point, Polygon
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
url = 'https://www.dropbox.com/s/za6yp2mz6p560o1/uber_trips_2014_nta.csv.gz?dl=1'
uber_trips_2014_nta=pd.read_csv(url, compression='gzip')

In [3]:
uber_trips_2014_nta.head()

,Unnamed: 0,pickup_datetime,pickup_latitude,pickup_longitude,base,pickup_point,pickup_nta
0,0,4/1/14 0:11,40.7690,-73.9549,B02512,POINT (40.769 -73.95489999999999),NaN
1,1,4/1/14 0:17,40.7267,-74.0345,B02512,POINT (40.7267 -74.03449999999999),NaN
2,2,4/1/14 0:21,40.7316,-73.9873,B02512,POINT (40.7316 -73.9873),MN22
3,3,4/1/14 0:28,40.7588,-73.9776,B02512,POINT (40.7588 -73.9776),MN17
4,4,4/1/14 0:33,40.7594,-73.9722,B02512,POINT (40.7594 -73.9722),MN19


In [4]:
uber2015=pd.read_csv("C:/Users/Rodrigo Bernal/Desktop/DS4A_workspace/Datathon/Dataset/Dataset/uber_trips_2015.csv")

In [5]:
url = 'https://www.dropbox.com/s/pjovxyr8w6x6y5x/zones.csv?dl=1'
zones=pd.read_csv(url)

In [6]:
zones.head()

,location_id,borough,zone,service_zone,nta_code
0,1,EWR,Newark Airport,EWR,NJ01
1,2,Queens,Jamaica Bay,Boro Zone,QN61
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,BX31
3,4,Manhattan,Alphabet City,Yellow Zone,MN22
4,5,Staten Island,Arden Heights,Boro Zone,SI48


In [7]:
uber_trips_2014_nta = pd.merge(uber_trips_2014_nta,zones, how='left' ,left_on=['pickup_nta'],right_on=['nta_code'])

In [8]:
uber_trips_2015_nta = pd.merge(uber2015,zones, how='left' ,left_on=['pickup_location_id'],right_on=['location_id'])

In [9]:
uber_trips_2014_nta.head()
del uber_trips_2014_nta['Unnamed: 0']
del uber_trips_2014_nta['pickup_latitude']
del uber_trips_2014_nta['pickup_longitude']
del uber_trips_2014_nta['base']
del uber_trips_2014_nta['pickup_point']
del uber_trips_2014_nta['pickup_nta']
#,'pickup_latitude','pickup_longitude','base','pickup_point','pickup_nta']

In [10]:
uber_trips_2014_nta.head()

,pickup_datetime,location_id,borough,zone,service_zone,nta_code
0,4/1/14 0:11,NaN,NaN,NaN,NaN,NaN
1,4/1/14 0:17,NaN,NaN,NaN,NaN,NaN
2,4/1/14 0:21,4.0,Manhattan,Alphabet City,Yellow Zone,MN22
3,4/1/14 0:21,79.0,Manhattan,East Village,Yellow Zone,MN22
4,4/1/14 0:28,161.0,Manhattan,Midtown Center,Yellow Zone,MN17


In [11]:
uber_trips_2015_nta.head()
del uber_trips_2015_nta['pickup_location_id']
del uber_trips_2015_nta['dispatch_base']
del uber_trips_2015_nta['affiliate_base']

In [12]:
uber_trips_2015_nta.head()

,pickup_datetime,location_id,borough,zone,service_zone,nta_code
0,2015-05-17 09:47:00,141.0,Manhattan,Lenox Hill West,Yellow Zone,MN31
1,2015-05-17 09:47:00,65.0,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,BK38
2,2015-05-17 09:47:00,100.0,Manhattan,Garment District,Yellow Zone,MN13
3,2015-05-17 09:47:00,80.0,Brooklyn,East Williamsburg,Boro Zone,BK90
4,2015-05-17 09:47:00,90.0,Manhattan,Flatiron,Yellow Zone,MN13


In [13]:
uber_trips=uber_trips_2014_nta.append(uber_trips_2015_nta, ignore_index=True)
del uber_trips_2014_nta
del uber_trips_2015_nta

In [14]:
uber_trips["pickup_datetime"]=pd.to_datetime(uber_trips["pickup_datetime"])
uber_trips['date'] = uber_trips['pickup_datetime'].dt.date
uber_trips['month_year'] = uber_trips['pickup_datetime'].dt.to_period('M')
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
uber_trips['weekday']=uber_trips["pickup_datetime"].dt.dayofweek
#uber2015['daytype'] = np.where(uber2015['weekday']>=5, 'weekend', 'weekday')
#uber2015['weekday']=uber2015["date"].dt.dayofweek.map(dayOfWeek)
uber_trips['hour']=uber_trips["pickup_datetime"].dt.hour

In [15]:
uber_trips.head()

,pickup_datetime,location_id,borough,zone,service_zone,nta_code,date,month_year,weekday,hour
0,2014-04-01 00:11:00,NaN,NaN,NaN,NaN,NaN,2014-04-01,2014-04,1,0
1,2014-04-01 00:17:00,NaN,NaN,NaN,NaN,NaN,2014-04-01,2014-04,1,0
2,2014-04-01 00:21:00,4.0,Manhattan,Alphabet City,Yellow Zone,MN22,2014-04-01,2014-04,1,0
3,2014-04-01 00:21:00,79.0,Manhattan,East Village,Yellow Zone,MN22,2014-04-01,2014-04,1,0
4,2014-04-01 00:28:00,161.0,Manhattan,Midtown Center,Yellow Zone,MN17,2014-04-01,2014-04,1,0


In [16]:
url = 'https://www.dropbox.com/s/za6yp2mz6p560o1/uber_trips_2014_nta.csv.gz?dl=1'
green_trips=pd.read_csv(url, compression='gzip')

In [17]:
del green_trips['Unnamed: 0']
del green_trips['pickup_latitude']
del green_trips['pickup_longitude']
del green_trips['base']
del green_trips['pickup_point']
green_trips.head()

,pickup_datetime,pickup_nta
0,4/1/14 0:11,NaN
1,4/1/14 0:17,NaN
2,4/1/14 0:21,MN22
3,4/1/14 0:28,MN17
4,4/1/14 0:33,MN19


In [ ]:
green_trips=pd.merge(green_trips,zones, how='left' ,left_on=['pickup_nta'],right_on=['nta_code'])
del green_trips['pickup_nta']

In [ ]:
green_trips["pickup_datetime"]=pd.to_datetime(green_trips["pickup_datetime"])
green_trips['date'] = green_trips['pickup_datetime'].dt.date
green_trips['month_year'] = green_trips['pickup_datetime'].dt.to_period('M')
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
green_trips['weekday']=green_trips["pickup_datetime"].dt.dayofweek
#uber2015['daytype'] = np.where(uber2015['weekday']>=5, 'weekend', 'weekday')
#uber2015['weekday']=uber2015["date"].dt.dayofweek.map(dayOfWeek)
green_trips['hour']=green_trips["pickup_datetime"].dt.hour

In [ ]:
url = 'https://www.dropbox.com/s/k4amamqkqf7ap92/yellow_trips_nta.csv.gz?dl=1'
yellow_trips=pd.read_csv(url, compression='gzip')

In [ ]:
del yellow_trips['dropoff_datetime']
del yellow_trips['dropoff_longitude']
del yellow_trips['dropoff_latitude']
del yellow_trips['passenger_count']
del yellow_trips['trip_distance']
del yellow_trips['total_amount']
del yellow_trips['dropoff_point']
del yellow_trips['dropoff_nta']
del yellow_trips['pickup_point']
del yellow_trips['Unnamed: 0']
del yellow_trips['pickup_longitude']
del yellow_trips['pickup_latitude']
yellow_trips.head()

In [ ]:
yellow_trips=pd.merge(yellow_trips,zones, how='left' ,left_on=['pickup_nta'],right_on=['nta_code'])
del yellow_trips['pickup_nta']

In [ ]:
yellow_trips["pickup_datetime"]=pd.to_datetime(yellow_trips["pickup_datetime"])
yellow_trips['date'] = yellow_trips['pickup_datetime'].dt.date
yellow_trips['month_year'] = yellow_trips['pickup_datetime'].dt.to_period('M')
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
yellow_trips['weekday']=yellow_trips["pickup_datetime"].dt.dayofweek
#uber2015['daytype'] = np.where(uber2015['weekday']>=5, 'weekend', 'weekday')
#uber2015['weekday']=uber2015["date"].dt.dayofweek.map(dayOfWeek)
yellow_trips['hour']=yellow_trips["pickup_datetime"].dt.hour

In [ ]:
uber_trips_manhattan=uber_trips[uber_trips['borough']=='Manhattan']
yellow_trips_manhattan=yellow_trips[yellow_trips['borough']=='Manhattan']
green_trips_manhattan=green_trips[green_trips['borough']=='Manhattan']
uber_trips_manhattan['class']='Uber'
yellow_trips_manhattan['class']='Yellow'
green_trips_manhattan['class']='Green'

In [ ]:
manhattan_trips=uber_trips_manhattan.append([yellow_trips_manhattan, green_trips_manhattan])

In [ ]:
manhattan_trips.head()

In [ ]:
manhattan_trips.groupby(['weekday','zone']).size(), columns=['number_of_trips']

In [ ]:
manhattan_trips_by_dayzone=pd.DataFrame(manhattan_trips.groupby(['weekday','zone']).size(), columns=['number_of_trips'])
manhattan_trips_by_dayzone.reset_index(inplace=True)
manhattan_trips_by_hourzone=pd.DataFrame(manhattan_trips.groupby(['hour','zone']).size(), columns=['number_of_trips'])
manhattan_trips_by_hourzone.reset_index(inplace=True)

In [ ]:
pivot_df = MAN_ubertrips_by_dayzone.pivot('zone','weekday','number_of_trips')
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(pivot_df, cmap="Reds", ax=ax);
#ax.set_title('PEC Electric Power Sector');

In [ ]:
pivot_df = MAN_ubertrips_by_hourzone.pivot('zone','hour','number_of_trips')
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(pivot_df, cmap="Reds", ax=ax);
#ax.set_title('PEC Electric Power Sector');